# Majority voting classifier

In [ ]:
outfile = 'majority_voting.csv'

GB builds an additive model in a forward stage-wise fashion; it allows for the optimization of arbitrary differentiable loss functions. In each stage n_classes_ regression trees are fit on the negative gradient of the binomial or multinomial deviance loss function. Binary classification is a special case where only a single regression tree is induced.

In [1]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

import numpy as np
import pandas as pd
from helper_methods import load_data
from sklearn.metrics import log_loss
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold

In [2]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()

In [3]:
df_train, y_train, ids_train, features_set = load_data(train=True, engineering=True, standardizer=stdsc)

In [4]:
StatifiedCV = StratifiedKFold(y_train, n_folds=10, shuffle=True)

### Hyperparameter and feature set tuning

For each set of features, use stratified cross validation to find the optimal hyperparameters:

In [5]:
scores = []
lr = LogisticRegression()
rf = RandomForestClassifier(n_estimators=100)
gb = GradientBoostingClassifier()
clf = VotingClassifier(estimators=[('lr', lr), ('rf', rf), ('gb', gb)], voting='soft')

param_grid = dict(lr__C=np.logspace(-3, 3, base=10, num=7),
                  lr__penalty=['l1', 'l2'],
                  rf__criterion=['gini', 'entropy'],
                  gb__learning_rate=np.logspace(-3, 0, base=10, num=4),
                  gb__n_estimators=[25, 50, 100],
                  gb__max_depth=[1, 2, 3])
for i, features in enumerate(features_set):
    grid = GridSearchCV(clf, param_grid, cv=StatifiedCV, scoring='log_loss')
    grid.fit(X=df_train[features].values, y=y_train)
    scores.append((-grid.best_score_, i, grid.best_params_))
    print i, -grid.best_score_, grid.best_params_

KeyboardInterrupt: 

### Set the optimal hyperparameters and feature set; refit the model to the entire training set

In [ ]:
scores.sort()
optimal_features = features_set[scores[0][1]]
optimal_params = scores[0][2]
X_train = df_train[optimal_features].values
optimal_clf = clf.set_params(**optimal_params).fit(X_train, y_train)
print 'Optimal score: %.2f' % scores[0][0]
print 'Optimal features: ', optimal_features
print 'Optimal parameters: ', optimal_params

### Load the test data, predict the probabilities and write to csv

In [ ]:
df_test, y_dummy, ids_test, features_set_dummy = load_data(train=False, engineering=True, standardizer=stdsc)

In [ ]:
X_test = df_test[optimal_features].values
y_pred_prob = optimal_clf.predict_proba(X_test)[:,1]
df_out = pd.DataFrame({'':ids_test, 'Made Donation in March 2007':y_pred_prob})
df_out.to_csv(outfile, index=False, header=True)

# Evaluation of the optimal classifier

### Learning curve

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('halverson')

from sklearn.learning_curve import learning_curve

train_sizes, train_scores, test_scores = learning_curve(optimal_clf, X_train, y_train, train_sizes=np.linspace(0.1, 1, 10), cv=10)
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(train_sizes, train_mean, color='b', marker='o', label='Train accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='b')
plt.plot(train_sizes, test_mean, color='g', marker='o', label='Validation accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='g')

plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.grid()
plt.ylim(0.3, 1)

The learning curve for the optimal set of parameters and features is shown above. We see that the model is well fit. The shaded region indicate the standard deviation of the accuracy scores.

### Confusion matrix

In [ ]:
from sklearn.cross_validation import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred_cv = cross_val_predict(optimal_clf, X_train, y_train, cv=StatifiedCV)
confmat = confusion_matrix(y_true=y_train, y_pred=y_train_pred_cv)
print confmat

The function cross_val_predict has a similar interface to cross_val_score, but returns for each element in the input the prediction that was obtained for that element when it was in the test set. It does not offer a predict_proba method yet but this is under discussion.

In [ ]:
fig, ax = plt.subplots(figsize=(3, 3))
ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confmat.shape[0]):
    for j in range(confmat.shape[1]):
        ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
plt.xlabel('Predicted label')
plt.ylabel('True label')

### Metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

print 'Accuracy:  %.2f' % accuracy_score(y_true=y_train, y_pred=y_train_pred_cv)
print 'Recall:    %.2f' % recall_score(y_true=y_train, y_pred=y_train_pred_cv)
print 'Precision: %.2f' % precision_score(y_true=y_train, y_pred=y_train_pred_cv)
print 'F1 score:  %.2f' % f1_score(y_true=y_train, y_pred=y_train_pred_cv)
print 'Log loss:  %.2f' % log_loss(y_true=y_train, y_pred=y_train_pred_cv)

### Permutation test score

In order to test if a classification score is significant, a technique in repeating the classification procedure after randomizing, permuting, the labels. The p-value is then given by the percentage of runs for which the score obtained is greater than the classification score obtained in the first place.

In [ ]:
from sklearn.cross_validation import permutation_test_score

score, permutation_scores, pvalue = permutation_test_score(optimal_clf, X_train, y_train, cv=StatifiedCV, n_permutations=100, scoring='accuracy')
plt.figure(figsize=(14,5))
plt.hist(permutation_scores, 20, label='Permutation scores')
plt.plot(2 * [score], plt.ylim(), '--g', linewidth=2, label='Classification score (pvalue {:.2f})'.format(pvalue))
plt.plot(2 * [1. / 2], plt.ylim(), '--r', linewidth=2, label='Random guessing')
plt.legend(loc='upper center')
plt.xlabel('Accuracy')
plt.ylabel('Count')
plt.xlim(0.45, 0.8)